# **Over- und Underfitting**

Dieses Colab-Notebook zeigt die Auswirkungen von Over- und Underfitting anhand von Testdaten sowie der Funktionswahl.

Quelle: https://github.com/WillKoehrsen/Data-Analysis/blob/master/over_vs_under/Over%20vs%20Under%20Fitting%20Example.ipynb

## Imports

Wir importieren zuerst die benötigten Libraries

In [ ]:
# Numpy und Pandas
import numpy as np
import pandas as pd

# Scikit-Learn für Machine Learning
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error

# Matplotlib für Plots
import matplotlib
import matplotlib.pyplot as plt
%matplotlib inline

# Matplotlib-Parameter
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['figure.titlesize'] = 16
matplotlib.rcParams['figure.figsize'] = [9, 7]

## Funktion definieren

Als erstes müssen wir eine Funktion definieren, um welche sich nachher unsere Daten streuen werden. Wir verwenden in unserem Beispiel folgende Funktion:

$$y = f(x) = \sin(1.2 \cdot x \cdot \pi)$$

Anschliessend generieren wir einige Testdaten und fügen etwas Rauschen (Streuung dazu)

In [ ]:
# Verteilungs-Initialisierung für die Random-Funktion
np.random.seed(42)

# Funktion definieren - Diese versucht das Modell anschliessend nachzubilden
def true_gen(x):
    y = np.sin(1.2 * x * np.pi)
    return(y)

# x und y Werte generieren. bei den y-Werten werden noch Abweichungen hinzugefügt.
x = np.sort(np.random.rand(120))
y = true_gen(x) + 0.1 * np.random.randn(len(x))

**Trainings-Daten generieren**

Nun generieren wir zufällige Trainings- und Testdaten. Für ein besseres Verständnis, visualisieren wir anschliessend die Daten.

In [ ]:
# Trainings-Werte zufällig erstellen
random_ind = np.random.choice(list(range(120)), size = 120, replace=False)
xt = x[random_ind]
yt = y[random_ind]

# Training- und Testdaten erstellen
train = xt[:int(0.7 * len(x))]
test = xt[int(0.7 * len(x)):]

y_train = yt[:int(0.7 * len(y))]
y_test = yt[int(0.7 * len(y)):]

# Die Funktion modellieren
x_linspace = np.linspace(0, 1, 1000)
y_true = true_gen(x_linspace)

In [ ]:
# Daten plotten
plt.plot(train, y_train, 'ko', label = 'Train');
plt.plot(test, y_test, 'ro', label = 'Test')
plt.plot(x_linspace, y_true, 'b-', linewidth = 2, label = 'Reale Funktion')
plt.legend()
plt.xlabel('x'); plt.ylabel('y'); plt.title('Daten');

Die Grafik zeigt uns anschaulich, dass die generierten Daten sich um unsere reale Sinusfunktion legen (aber doch eine gewisse Zufälligkeit aufweisen)

## Polynominale Funktion

Da wir eine Sinus-Kurve abbilden wollen, müssen wir zwangsläufig auf eine Polynom-Funktion ausweichen (Polynome sind Funktionen n-ten Grades)

Wir definieren dazu ein Code-Snippet, welches ein Machine-Learning anhand des mitgegebenen Grades erstellt. Mittels dem Modell können wir anschliessend aufzeigen, wie sich Over- und Underfitting verhält.

In [ ]:
def fit_poly(train, y_train, test, y_test, degrees, plot='train', return_scores=False):

    # Create a polynomial transformation of features
    features = PolynomialFeatures(degree=degrees, include_bias=False)

    # Reshape training features for use in scikit-learn and transform features
    train = train.reshape((-1, 1))
    train_trans = features.fit_transform(train)

    # Create the linear regression model and train
    model = LinearRegression()
    model.fit(train_trans, y_train)

    # Calculate the cross validation score
    cross_valid = cross_val_score(model, train_trans, y_train, scoring='neg_mean_squared_error', cv = 5)

    # Training predictions and error
    train_predictions = model.predict(train_trans)
    training_error = mean_squared_error(y_train, train_predictions)

    # Format test features
    test = test.reshape((-1, 1))
    test_trans = features.fit_transform(test)

    # Test set predictions and error
    test_predictions = model.predict(test_trans)
    testing_error = mean_squared_error(y_test, test_predictions)

    # Find the model curve and the true curve
    x_curve = np.linspace(0, 1, 100)
    x_curve = x_curve.reshape((-1, 1))
    x_curve_trans = features.fit_transform(x_curve)

    # Model curve
    model_curve = model.predict(x_curve_trans)

    # True curve
    y_true_curve = true_gen(x_curve[:, 0])

    # Plot observations, true function, and model predicted function
    if plot == 'train':
        plt.plot(train[:, 0], y_train, 'ko', label = 'Observations')
        plt.plot(x_curve[:, 0], y_true_curve, linewidth = 4, label = 'True Function')
        plt.plot(x_curve[:, 0], model_curve, linewidth = 4, label = 'Model Function')
        plt.xlabel('x'); plt.ylabel('y')
        plt.legend()
        plt.ylim(-1, 1.5); plt.xlim(0, 1)
        plt.title('{} Degree Model on Training Data'.format(degrees))
        plt.show()

    elif plot == 'test':
        # Plot the test observations and test predictions
        plt.plot(test, y_test, 'o', label = 'Test Observations')
        plt.plot(x_curve[:, 0], y_true_curve, 'b-', linewidth = 2, label = 'True Function')
        plt.plot(test, test_predictions, 'ro', label = 'Test Predictions')
        plt.ylim(-1, 1.5); plt.xlim(0, 1)
        plt.legend(), plt.xlabel('x'), plt.ylabel('y'); plt.title('{} Degree Model on Testing Data'.format(degrees)), plt.show();

    # Return the metrics
    if return_scores:
        return training_error, testing_error, -np.mean(cross_valid)

# **Modelle trainieren**

## Polynom 1. Grades (Lineare Funktion) -> Underfitting
Nun können wir als erstes Beispiel eine Lineare Funktion für das Model verwenden. Da wir eine Sinus-Kurve haben, ist es natürlich nicht möglich, diese Daten real abzubilden. In diesem Fall haben wir es mit Underfitting zu tun.

In [ ]:
fit_poly(train, y_train, test, y_test, degrees = 1, plot='train')

Wenn wir die Prediction ausgeben, sehen wir, wo sich unsere Testdaten befinden werden (kleiner Tipp: Das Model bildet die polynominale Funktion ab)

In [ ]:
fit_poly(train, y_train, test, y_test, degrees = 1, plot='test')

## Polynom 25. Grades -> Overfitting
Im nächsten Beispiel trainieren wir das Model mit einem 25. gradigen Polynom, um das Overfitting (oder auswendiglernen der Daten) zu erzeugen.

In [ ]:
fit_poly(train, y_train, test, y_test, plot='train', degrees = 25)

Wir sehen hier bereits in der Model-Funktion, dass das Modell die Daten mehr oder weniger auswendig lernt. Daher wird dieses Modell auch eine hohe Genauigkeit aufweisen.

In [ ]:
fit_poly(train, y_train, test, y_test, degrees=25, plot='test')

## Polynom 5. Grades -> Balanciertes Modell
Wir haben nun die zwei extreme gesehen. Das Ziel ist es, ein Modell zu erstellen, welches die Diversität der Daten gut abbilden kann, aber nicht zu extrem den Daten folgt. Im nächsten Versuch werden wir ein Polynom des 5. Grades trainieren, welches einen guten Mittelwert zeigt.

In [ ]:
fit_poly(train, y_train, test, y_test, plot='train', degrees = 5)

In [ ]:
fit_poly(train, y_train, test, y_test, degrees=5, plot='test')

Anhand der Testdaten sehen wir, dass das Modell sehr gut der echten Funktion folgt.

# **Cross Validation**
Wir müssen nun für unser Modell eien Balance finden, damit das Modell nicht over, aber auch nicht underfittet. Hier bietet sich Cross Validation an. Hierbei können wir die Fehlerrate pro Grad des Modells anzeigen. Anhand diesem können wir den optimalen Grad unseres Polynomes bestimmen.

In [ ]:
# Grad-Range des Polynomes wählen (in diesem Beispiel: 1 - 40)
degrees = [int(x) for x in np.linspace(1, 40, 40)]

# DataFrame definieren (für Resultate)
results = pd.DataFrame(0, columns = ['train_error', 'test_error', 'cross_valid'], index = degrees)

# Für jeden Grad trainieren und testen
for degree in degrees:
    degree_results = fit_poly(train, y_train, test, y_test, degree, plot=False, return_scores=True)
    results.loc[degree, 'train_error'] = degree_results[0]
    results.loc[degree, 'test_error'] = degree_results[1]
    results.loc[degree, 'cross_valid'] = degree_results[2]

Nun können die zehn niedrigsten Cross Validation Fehler ausgegeben werden. Hier sehen wir, dass wir gute Werte mit dem vierten Grad haben

In [ ]:
print('10 Lowest Cross Validation Errors\n')
train_eval = results.sort_values('cross_valid').reset_index(level=0).rename(columns={'index': 'degrees'})
train_eval.head(10)

Wir können die das Verhalten auch grafisch ausgeben, dort sehen wir, wie das Modell sich mit den einzelnen Graden verhält.

In [ ]:
plt.plot(results.index, results['cross_valid'], 'go-', ms=6)
plt.xlabel('Degrees'); plt.ylabel('Cross Validation Error'); plt.title('Cross Validation Results');
plt.ylim(0, 0.2);
print('Minimum Cross Validation Error occurs at {} degrees.\n'.format(int(np.argmin(results['cross_valid'])) + 1))

# **Finales Model**
Wir haben vorher gesehen, dass unser Modell bei einem Polynom im 3. Grad am besten performt. Somit können wir die Parameter entsprechend setzen

In [ ]:
fit_poly(train, y_train, test, y_test, degrees=3, plot='train')

In [ ]:
fit_poly(train, y_train, test, y_test, degrees=4, plot='test')

# **Modelle evaluieren**
Um die Modelle zu evaluieren, können wir die Training und Testfehlerrate ausgeben und anschliessend miteinander vergleichen

## Quantitativer Vergleich
Wir sehen in der nachfolgenden Tabelle, dass die Trainingsfehler kleiner werden, je höher der Grad wird. Das ist ein deutliches Zeichen für Overfitting, da das Modell die Daten auswendig lernt.

In [ ]:
print('10 Lowest Training Errors\n')
train_eval = results.sort_values('train_error').reset_index(level=0).rename(columns={'index': 'degrees'})
train_eval.loc[:,['degrees', 'train_error']] .head(10)

In dieser Tabelle sehen wir, dass der Testfehler beim 5. Grad am niedrigsten ist. Somit haben wir bei diesem Grad einen guten Startpunkt

In [ ]:
print('10 Lowest Testing Errors\n')
train_eval = results.sort_values('test_error').reset_index(level=0).rename(columns={'index': 'degrees'})
train_eval.loc[:,['degrees', 'test_error']] .head(10)

## Visueller Vergleich
Wir können die Werte auch visuell Ausgben, hier sehen wir, dass das Modell beim 5. Grad im Verhältnis zum Training- und Testfehler optimal performt.

In [ ]:
plt.plot(results.index, results['train_error'], 'b-o', ms=6, label = 'Training Error')
plt.plot(results.index, results['test_error'], 'r-*', ms=6, label = 'Testing Error')
plt.legend(loc=2); plt.xlabel('Degrees'); plt.ylabel('Mean Squared Error'); plt.title('Training and Testing Curves');
plt.ylim(0, 0.05); plt.show()

print('\nMinimum Training Error occurs at {} degrees.'.format(int(np.argmin(results['train_error']))))
print('Minimum Testing Error occurs at {} degrees.\n'.format(int(np.argmin(results['test_error']))))